<a href="https://colab.research.google.com/github/raptor-ml/docs/blob/master/docs/guides/getting-started-with-labsdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[↵ Back to the Docs](https://raptor.ml)

<img src="https://raptor.ml/img/logo.svg" height="200" />

# 🦖 LabSDK

Using the LabSDK, Data-scientists can build features(that can run on production) directly from the notebook while developing your model.

When you're done, you can "export" the features as Kubernetes manifests, and deploy them like any other service in your cluster. This way, you can benefit from the "serverless approach", and **focus on the business-logic**, while Raptor is taking care of the production concerns.

## 🧐 Getting started
In this quickstart tutorial, we'll build a model that predicts the probability of closing a deal.

Our CRM allow us to track every email communication, and the history of previous deals for each customer. We'll use this data sources to predict whether the customer is ready for closure or not.

To do that, we're going to build a few features from the data:
1. `emails_10h` - the amount of email exchanges over the last 10 hours
1. `deals_10h[sum]` - the sum of the deals of the last 10 hours
1. `emails_deals` - the rate between the emails in the last 10 hours (`emails_10h`) and the avarage of the deals in the last 10 hours (`deals_10h[avg]`)

## ⚡️ Installing the SDK
Yalla, let's go! In the following two blocks, we install the LabSDK and import it.

In [1]:
!pip install raptor-labsdk -U --quiet
import raptor
from raptor.stub import *  #<-- this prevents the IDE/Notebookfrom detecting PyExp built-in as errors

## ✍️ Writing our first features
Our first feature is calculating how many emails an account got over the last 10 hours.

In the next section we'll "replay" our historical data on this function, but for now - we are only specifying that it uses the `emails` data-connector(which DevOps needs to configure for us on produciton).

In [2]:
@raptor.register(int, staleness='10h')
@raptor.connector("emails")  #<-- we are decorating our feature with our production data-connector! 😎 
@raptor.aggr([raptor.AggrFn.Count], granularity='1m')
def emails_10h(**req: RaptorRequest):
    """email over 10 hours"""
    return 1

---
> ## 😎 *Cool tip* 
>
> Although it's looks like a regular python, it's not 😲
>
> The feature definition above is actually written in [PyExp](https://raptor.ml/docs/reference/pyexp), it's a dialect of Python and allow us to compile your code and run it production (in a safe and performant manner)
>
> [Learn more about PyExp »](https://raptor.ml/docs/reference/pyexp)

Let's create another feature that calculates various aggregations against the deal amount.

Notice that we're specified the `deals` data-connector here. In production, this data-connector will be used to retreive the "real" data.

In [3]:
@raptor.register(int, staleness='10h')
@raptor.connector("deals")
@raptor.aggr([raptor.AggrFn.Sum, raptor.AggrFn.Avg, raptor.AggrFn.Max, raptor.AggrFn.Min], granularity='1m')
def deals_10h(**req: RaptorRequest):
    """sum/avg/min/max of deal amount over 24 hours"""
    return req['payload']["amount"]

Now we can create a *derived feature* that defines the rate between these two features.

**💡Hint:** Notice that when querying a feature with aggregation, we need to specify the feature with the aggregation feature we want in the brackets (using the FQN notation).

In [4]:
@raptor.register('headless', staleness='-1', freshness='-1')
def emails_deals(**req: RaptorRequest):
    """emails/deal[avg] rate over 10 hours"""
    e, _ = get_feature("emails_10h[count]", req['entity_id'])
    d, _ = get_feature("deals_10h[avg]", req['entity_id'])
    if e == None or d == None:
        return None
    return e / d

And finally, prepare it as a data set:

In [5]:
@raptor.feature_set(register=True)
def deal_prediction():
    return "emails_10h[count]", "deals_10h[sum]", emails_deals

## 💾 Historical Replay
We can "replay" the historical records against our production-ready feature that we have written above for development purposes.

The SDK will run this code locally and allow us to iterate on it quickly.

In [6]:
!pip install pandas pyarrow -U --quiet
import pandas as pd

In [7]:
# first, calculate the "root" features
df = pd.read_parquet("https://gist.github.com/AlmogBaku/a1b331615eaf1284432d2eecc5fe60bc/raw/emails.parquet")
emails_10h.replay(df, entity_id_field="account_id")

df = pd.read_csv("https://gist.githubusercontent.com/AlmogBaku/a1b331615eaf1284432d2eecc5fe60bc/raw/deals.csv")
deals_10h.replay(df, entity_id_field="account_id")

# then, we can calculate the derrived features
emails_deals.replay(df, entity_id_field="account_id")

,timestamp,entity_id,fqn,value


---
> ℹ️ **Looking to run Replay at *scale*?** try [Raptor Enterprise](mailto:contact@raptor.ml) 🦖


## 🧠 Building our model
To use our set in for our model, we need to query it:

In [8]:
df = deal_prediction.historical_get(since='2020-1-1', until='2022-12-31')
df
# model.fit(df)

,timestamp,entity_id,emails_10h.default[count],deals_10h.default[sum],emails_deals.default
0,2022-01-01 12:00:00+00:00,msft,1.0,458.0,0.002183
1,2022-01-01 12:00:00+00:00,tesla,1.0,8837.0,0.000113
2,2022-01-01 12:20:00+00:00,tesla,2.0,103502.0,0.000039
3,2022-01-01 13:10:00+00:00,msft,2.0,1727.0,0.002316
4,2022-01-01 13:20:00+00:00,msft,3.0,3063.0,0.002938
5,2022-01-01 13:40:00+00:00,tesla,3.0,109966.0,0.000082
6,2022-01-01 14:00:00+00:00,msft,4.0,7599.0,0.002106
7,2022-01-01 14:10:00+00:00,msft,5.0,14583.0,0.001714
8,2022-01-01 14:20:00+00:00,msft,6.0,23132.0,0.001556
9,2022-01-01 14:30:00+00:00,msft,7.0,27775.0,0.001764


## ☁️ Deployment
That's the fun part! 🤗 Making our features run at scale in production couldn't be easier.

We only need to deploy our feature definitions to the Raptor Platform.
You can do that with a preferred CI/CD of your choice, manually via `kubectl` or directly from your Jupyter Notebook(but that's not recommended for real-production environments 🤪)

### Manifest deployment (only use this for production)
We *copy-and-paste* the generated manifests to git, and use the organization's preferred method for deploying Kubernetes manifests (i.e. gitops, jenkins, kustomize, helm, etc.)

In [9]:
raptor.manifests(print_manifests=True)

apiVersion: k8s.raptor.ml/v1alpha1
kind: Feature
metadata:
  name: emails-10h
  annotations:
    a8r.io/description: email over 10 hours
spec:
  primitive: int
  freshness: 1m
  staleness: 10h
  connector:
    name: emails
  builder:
    aggr:
    - 'count'
    aggr_granularity: 1m
    pyexp: |-
      def emails_10h(**req):
          'email over 10 hours'
          return 1
---
apiVersion: k8s.raptor.ml/v1alpha1
kind: Feature
metadata:
  name: deals-10h
  annotations:
    a8r.io/description: sum/avg/min/max of deal amount over 24 hours
spec:
  primitive: int
  freshness: 1m
  staleness: 10h
  connector:
    name: deals
  builder:
    aggr:
    - 'sum'
    - 'avg'
    - 'max'
    - 'min'
    aggr_granularity: 1m
    pyexp: |-
      def deals_10h(**req):
          'sum/avg/min/max of deal amount over 24 hours'
          return req['payload']['amount']
---
apiVersion: k8s.raptor.ml/v1alpha1
kind: Feature
metadata:
  name: emails-deals
  annotations:
    a8r.io/description: emails/deal[avg

### Direct deployment (for local development)
Alternatively, we can just deploy it on our own directly from python:

1. Make sure you've installed and configured `kubectl` on the notebook device.
2. Then, you can create and upload your manifests directly from the notebook
```python
manifests = raptor.manifests()
!echo "$manifests" | kubectl apply -f -
```

>> **💡 Hint:** We didn't include these blocks as executable since you need to configure your cluster.

## 🪄 Ta-dam!
**From now on**, our cluster will collect and build features in production and record the values for historical purposes (so you'll be able to retrain against the production data).

[**🔗 Learn more about what you can do with Raptor at the official docs**](https://raptor.ml)
